# Google colab version

In [ ]:
# Clone the repository
!git clone https://github.com/jonasricker/aeroblade.git

# Navigate into the cloned directory
%cd aeroblade

# Install a specific Python version using a more stable method for Colab
# Note: This will restart the kernel.
!pip install -q virtualenv
!virtualenv -p python3.10 aeroblade_env
!source aeroblade_env/bin/activate

In [ ]:
# Install the core and pip-managed packages.
# We are now pinning a compatible version of peft.
!pip install -v --no-cache-dir --no-input \
    --extra-index-url https://download.pytorch.org/whl/cu128 \
    torch==2.8.0+cu128 torchvision==0.23.0+cu128 \
    peft==0.7.0 \
    scipy numpy \
    scikit-learn scikit-image \
    seaborn tqdm \
    networkx pyarrow \
    diffusers==0.25.1 huggingface-hub==0.25.2 \
    requests PyYAML filelock \
    pillow opencv-python \
    jupyter-client ipykernel ipython jupyter-core \
    clip-interrogator==0.6.0 joblib==1.5.2 lpips==0.1.4 pyiqa==0.1.14.1
    
# Install the project in editable mode
!pip install -e .

# Locally run version. You may need to run this as administrator.

In [ ]:
# # Clone the repository
# !git clone https://github.com/jonasricker/aeroblade.git

# # Navigate into the cloned directory
# %cd aeroblade

c:\PersonalStuff\Monash\Sem3\FIT5230MaliciousAI\Assignments\cloned_repos\aeroblade


In [ ]:
# import sys
# import subprocess
# import shutil

# # Find the full path to the conda executable
# conda_path = shutil.which("conda")
# if conda_path is None:
#     print("Error: conda executable not found. Please ensure conda is installed and in your system PATH.")
#     # You might want to exit the script here.
# else:
#     print(f"Found conda at: {conda_path}")

#     # List of conda-managed packages
#     conda_packages = [
#         'python=3.10',
#         'numpy=1.26.3',
#         'pandas=2.1.4',
#         'scipy=1.12.0',
#         'matplotlib=3.8.2',
#         'scikit-learn=1.4.0',
#         'scikit-image=0.22.0',
#         'seaborn=0.13.2',
#         'tqdm=4.66.1',
#         'networkx=3.2.1',
#         'pyarrow=15.0.0',
#         'pip'
#     ]

#     # Install conda packages
#     print("Installing conda packages...")
#     try:
#         subprocess.run([conda_path, 'install', '-y', '-c', 'conda-forge'] + conda_packages, check=True)
#         print("Successfully installed conda packages.")
#     except subprocess.CalledProcessError as e:
#         print(f"Failed to install conda packages: {e}")

Found conda at: C:\Users\adria\miniconda3\condabin\conda.BAT
Installing conda packages...


In [ ]:
# import sys
# import subprocess

# # List of pip-managed packages
# pip_packages = [
#     '--extra-index-url', 'https://download.pytorch.org/whl/cu128',
#     'torch==2.8.0+cu128',
#     'torchvision==0.23.0+cu128',
#     'diffusers==0.25.1',
#     'huggingface-hub==0.25.2',
#     'transformers==4.37.2',
#     'requests==2.31.0',
#     'PyYAML==6.0.1',
#     'filelock==3.13.1',
#     'pillow==10.2.0',
#     'opencv-python==4.9.0.80',
#     'jupyter-client==8.6.0',
#     'ipykernel==6.28.0',
#     'ipython==8.20.0',
#     'jupyter-core==5.7.1',
#     'clip-interrogator==0.6.0',
#     'joblib==1.5.2',
#     'lpips==0.1.4',
#     'pyiqa==0.1.14.1'
# ]

# # Install pip packages
# print("Installing pip packages...")
# try:
#     subprocess.run([sys.executable, '-m', 'pip', 'install', '--no-cache-dir', '--no-input'] + pip_packages, check=True)
#     print("Successfully installed pip packages.")
# except subprocess.CalledProcessError as e:
#     print(f"Failed to install pip packages: {e}")

# # Replicates !pip install -e . Might need administrator access.
# print("Installing in editable mode...")
# try:
#     subprocess.run([sys.executable, '-m', 'pip', 'install', '-e', '.'], check=True)
#     print("Successfully installed in editable mode.")
# except subprocess.CalledProcessError as e:
#     print(f"Failed to install in editable mode: {e}")

Installing in editable mode...


# Run the scripts. 
This is the same whether on colab or locally run.
Input your adversarial images as png images into example_images, or specify the folder directory manually.

In [4]:
# Specify your image directory
# !python scripts/run_aeroblade.py --files-or-dirs path/to/img.png path/to/dir

# Runs on the images in example_images
!python scripts/run_aeroblade.py

Images saved to aeroblade_output\reconstructions\ed49694fbf045a034cac7c6ce4a44c4d\1.
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [on]
Loading model from: c:\Users\adria\miniconda3\envs\aeroblade_minimal_2\lib\site-packages\lpips\weights\v0.1\vgg.pth
Images saved to aeroblade_output\reconstructions\15ae2fefe620cab51104963e6df88aa6\1.
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [on]
Loading model from: c:\Users\adria\miniconda3\envs\aeroblade_minimal_2\lib\site-packages\lpips\weights\v0.1\vgg.pth
Images saved to aeroblade_output\reconstructions\48fb8d7f7fed1a356fcde0a9c4a317f2\1.
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [on]
Loading model from: c:\Users\adria\miniconda3\envs\aeroblade_minimal_2\lib\site-packages\lpips\weights\v0.1\vgg.pth
Images saved to aeroblade_output\reconstructions\db5772d924f212b9cd5c588e9bd60745\1.
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [on]
Loading model from: c:\Users\adr

c:\Users\adria\miniconda3\envs\aeroblade_minimal_2\lib\site-packages\diffusers\utils\outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
c:\Users\adria\miniconda3\envs\aeroblade_minimal_2\lib\site-packages\diffusers\utils\outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(

PROGRESS (compute_distances):   0%|          | 0/24 [00:00<?, ?it/s]c:\Users\adria\miniconda3\envs\aeroblade_minimal_2\lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Reconstructing with CompVis/stable-diffusion-v1-1.: 

In [ ]:
import pandas as pd
import os

# Input / output paths
input_csv = "aeroblade_output/distances.csv"
output_csv = "aeroblade_output/predictions.csv"

# Load distances
df = pd.read_csv(input_csv)

# Keep only 'max' rows (final aggregated distance per image)
df_max = df[df["repo_id"] == "max"].copy()

# Classification rule
def classify(distance):
    if 0 >= distance >= -0.02:
        return "fake"
    elif distance < -0.02:
        return "true"
    else:
        return "unknown"

df_max["prediction"] = df_max["distance"].apply(classify)

# Extract ground-truth labels from dir or file
def extract_label(path, filename):
    path_str = f"{path} {filename}".lower()
    if "real" in path_str:
        return "true"
    elif "fake" in path_str:
        return "fake"
    else:
        return "unknown"

df_max["ground_truth"] = df_max.apply(
    lambda row: extract_label(row["dir"], row["file"]), axis=1
)

# Select requested columns
df_pred = df_max[["dir", "image_size", "file", "prediction", "ground_truth"]]

# Save predictions
os.makedirs(os.path.dirname(output_csv), exist_ok=True)
df_pred.to_csv(output_csv, index=False)
print(f"Saved predictions to {output_csv}")

# --- Summary stats ---
# Count predictions overall
counts = df_pred["prediction"].value_counts(dropna=False)
print("\nPrediction counts (all images):")
print(counts.to_string())

# Evaluate only where ground-truth is known
valid = df_pred[df_pred["ground_truth"] != "unknown"]
if not valid.empty:
    accuracy = (valid["prediction"] == valid["ground_truth"]).mean()
    print(f"\nAccuracy (on {len(valid)} images with ground truth): {accuracy:.4f}")

    # Confusion matrix
    confusion = pd.crosstab(valid["ground_truth"], valid["prediction"],
                            rownames=["Actual"], colnames=["Predicted"], dropna=False)
    print("\nConfusion Matrix (only known ground truths):")
    print(confusion)
else:
    print("\n⚠️ No ground-truth labels detected (check filenames or directories).")


Saved predictions to aeroblade_output/predictions.csv
